In [1]:
import cv2
import dlib
import numpy as np
import time

In [2]:
# Initialize dlib's face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def get_3d_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        landmark_points = []
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmark_points.append((x, y))
        return landmark_points
    return None



In [ ]:
# Capture video from a file
cap = cv2.VideoCapture('/home/rafi-durrani/Documents/Hackathon/dataset/EAR_Selfie.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        frame = cv2.flip(frame, 0)
        landmarks = get_3d_landmarks(frame)
        if landmarks:
            for point in landmarks:
                cv2.circle(frame, point, 2, (255, 0, 0), -1)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
def calculate_EAR(eye_points):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = np.linalg.norm(np.array(eye_points[1]) - np.array(eye_points[5]))
    B = np.linalg.norm(np.array(eye_points[2]) - np.array(eye_points[4]))

    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = np.linalg.norm(np.array(eye_points[0]) - np.array(eye_points[3]))

    # compute the eye aspect ratio
    EAR = (A + B) / (2.0 * C)

    return EAR

In [4]:
def is_mouth_open(mouth_points):
    # Compute the euclidean distance between the upper and lower inner lips
    if mouth_points is not None:
        mouth_openness = np.linalg.norm(np.array(mouth_points[13]) - np.array(mouth_points[19]))  # Points 62 and 66
        # print(mouth_openness)
        # Define a threshold for mouth openness
        MOUTH_OPEN_THRESHOLD = 5  # This threshold will vary depending on the individual and camera quality
    else:
        pass
    return mouth_openness > MOUTH_OPEN_THRESHOLD

def mouth_movement(mouth_points, previous_mouth_points):
    if mouth_points is not None:
    # Compute the change in mouth openness between current and previous frames
        current_openness = np.linalg.norm(np.array(mouth_points[13]) - np.array(mouth_points[19]))
        previous_openness = np.linalg.norm(np.array(previous_mouth_points[13]) - np.array(previous_mouth_points[19]))

        # Define a threshold for determining significant mouth movement
        MOUTH_MOVEMENT_THRESHOLD = 1  # Adjust based on your specific requirements
        # print(abs(current_openness - previous_openness))
    else:
        pass
    return abs(current_openness - previous_openness) > MOUTH_MOVEMENT_THRESHOLD


In [5]:
def draw_landmarks(frame, landmarks):
    # Draw facial landmarks on the frame
    if landmarks is not None:
        for (x, y) in landmarks:
            cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)  # Green dots
    else:
        pass

    return frame

In [10]:
# Initialize previous mouth points
previous_mouth_points = None
cap = cv2.VideoCapture('/home/rafi-durrani/Documents/Hackathon/dataset/EAR.avi')

blink_rate = 20  # You could choose a value between 15 and 20
# expected_blinks = average_blink_rate_per_minute * video_length_minutes
filename  = "output.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(filename, fourcc, 20.0, (620, 430))
EAR_THRESHOLD = 0.111

MOUTH_MOVEMENT_SYNC_THRESHOLD = 5  # The threshold for lip sync discrepancies
MOUTH_OPENING_RATE_THRESHOLD = 3  # The expected frequency of mouth openings during speech

# Initialize counters and timers
blinking_counter = 0
mouth_open_counter = 0
start_time = time.time()


while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        # frame = cv2.flip(frame, 0)
        frame = cv2.resize(frame, (620, 430))
        landmarks = get_3d_landmarks(frame)
        
        frame_with_landmarks = draw_landmarks(frame, landmarks)
        
        # Assuming landmarks is a list of (x, y) tuples
        if landmarks is not None:
            left_eye_EAR = calculate_EAR(landmarks[36:42])
            right_eye_EAR = calculate_EAR(landmarks[42:48])
            blinking = left_eye_EAR < EAR_THRESHOLD and right_eye_EAR < EAR_THRESHOLD

        if blinking:
            # print("Blink detected")
            cv2.putText(frame_with_landmarks, "Blink detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            blinking_counter += 1

        if previous_mouth_points is not None:
            if landmarks is not None:
                mouth_open = is_mouth_open(landmarks[48:68])
                mouth_moving = mouth_movement(landmarks[48:68], previous_mouth_points)

            if mouth_open:
                # print("Mouth is open")
                cv2.putText(frame_with_landmarks, "Mouth is open", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                mouth_open_counter += 1

            if mouth_moving:
                #  print("Mouth movement detected")
                 cv2.putText(frame_with_landmarks, "Mouth movement detected", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        
        # Update the previous mouth points
        if landmarks is not None:
            previous_mouth_points = landmarks[48:68]

        cv2.imshow("Frame", frame_with_landmarks)
        out.write( frame_with_landmarks)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit the video display
            #int(1000 / (0.5 * 30))
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

elapsed_time = (time.time() - start_time) / 60
# print(elapsed_time)
# print(blinking_counter)
if elapsed_time > 0 and (blinking_counter / elapsed_time) < blink_rate:
    print("Blinking rate is abnormally low")
    
if elapsed_time > 0 and (mouth_open_counter / elapsed_time) < MOUTH_OPENING_RATE_THRESHOLD:
    print("Mouth opening rate during speech is abnormally low")
    


Blinking rate is abnormally low


In [ ]:


while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        landmarks = get_3d_landmarks(frame)
        if landmarks:
            for point in landmarks:
                cv2.circle(frame, point, 2, (255, 0, 0), -1)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# ... [existing code for facial landmark detection]

def analyze_landmarks(landmarks):
    # Placeholder function for analyzing landmarks
    # You would include logic here to analyze the movements and positions
    # of the landmarks to look for signs of deepfakes
    pass

def is_video_genuine(landmarks_list):
    # Analyze all frames and return a confidence score or decision
    # This is where you would integrate with a machine learning model
    # For now, we'll use a placeholder function
    for landmarks in landmarks_list:
        if not analyze_landmarks(landmarks):
            return False  # If any frame looks suspicious, flag the video
    return True

landmarks_all_frames = []
while cap.isOpened():
    # ... [existing frame capture code]
    if landmarks:
        landmarks_all_frames.append(landmarks)
    # ... [rest of the loop]

# After processing all frames
is_genuine = is_video_genuine(landmarks_all_frames)
print(f"Video is {'genuine' if is_genuine else 'fake'}")
